In [2]:
import tensorflow as tf
import sys
sys.path.append('/nas/longleaf/home/siyangj/NiftyNet/')
import niftynet as nn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import SimpleITK as sitk

data_path = '/proj/NIRAL/users/siyangj/myData/'
new_path = '/proj/NIRAL/users/siyangj/myData/clean/'
normal_path = '/proj/NIRAL/users/siyangj/myData/normal/'

In [2]:
im_num = np.arange(1,9)
suf1 = ['','-flip']
suf2 = {'-seg':'seg/','-T1-stripped':'T1/','-T2-stripped':'T2/'}
cur_dim = np.array([130,140,120])
tar_dim = np.array([136,144,120])
dif_dim = tar_dim - cur_dim
haf_dim = tar_dim / 2

In [28]:
for i in im_num:
    for s in suf1:
        for t,d in suf2.items():
            im_cur = sitk.ReadImage(data_path+str(i)+s+t+'.nrrd')
            ## Padding the image so that can be handled by 3D U-net
            im_pad = sitk.ConstantPad(im_cur,[0,2,3],[0,2,3],constant=0)
            sitk.WriteImage(im_pad,new_path+d+str(i)+s+'.nrrd')

In [35]:
for i in im_num:
    for s in suf1:
        im_T1 = sitk.ReadImage(new_path+'T1/'+str(i)+s+'.nrrd')
        im_T2 = sitk.ReadImage(new_path+'T2/'+str(i)+s+'.nrrd')
        im_C  = sitk.Compose(im_T1,im_T2)
        sitk.WriteImage(im_C,new_path+'T1T2/'+str(i)+s+'.nrrd')

In [ ]:
## Normalize each image
for i in im_num:
    for s in suf1:
        for t,d in suf2.items():
            im_cur = sitk.ReadImage(data_path+str(i)+s+t+'.nrrd')
            ## Padding the image so that can be handled by 3D U-net
            im_pad = sitk.ConstantPad(im_cur,[0,2,3],[0,2,3],constant=0)
            sitk.WriteImage(im_pad,new_path+d+str(i)+s+'.nrrd')

In [4]:
i=1;s='';

im_T1 = sitk.ReadImage(new_path+'T1/'+str(i)+s+'.nrrd')
im_T2 = sitk.ReadImage(new_path+'T2/'+str(i)+s+'.nrrd')
im_C  = sitk.Compose(im_T1,im_T2)

im_C.GetSize()

(120, 144, 136)

In [5]:
help(sitk.Normalize)

Help on function Normalize in module SimpleITK.SimpleITK:

Normalize(image1)
    Normalize(Image image1) -> Image
    
    
    
    Normalize an image by setting its mean to zero and variance to one.
    
    
    This function directly calls the execute method of NormalizeImageFilter in order to support a procedural API
    
    
    See:
     itk::simple::NormalizeImageFilter for the object oriented interface



In [6]:
tar_dim = [120,144,136]

crop= [([0,0,0],[0,0,0]),
       ([0,0,0],[0,0,0]),
       ([2,8,36],[2,8,18]),
       ([2,0,54],[2,0,0]),
       ([2,20,44],[2,13,0]),
       ([2,17,50],[2,16,14]),
       ([2,17,44],[2,16,0]),
       ([2,17,50],[2,16,14])]

pad = [([0,2,3],[0,2,3]),
       ([0,2,3],[0,2,3]),
       ([0,0,0],[0,0,0]),
       ([0,2,0],[0,2,0]),
       ([0,0,0],[0,0,0]),
       ([0,0,0],[0,0,0]),
       ([0,0,0],[0,0,0]),
       ([0,0,0],[0,0,0])]

In [7]:
for i in np.arange(1,9):
    pad_l = pad[i-1][0]
    pad_u = pad[i-1][1]
    crop_l = crop[i-1][0]
    crop_u = crop[i-1][1]

    for s in suf1:
        for t,d in suf2.items():
            im_cur = sitk.ReadImage(data_path+str(i)+s+t+'.nrrd')
            ## Padding the image so that can be handled by 3D U-net
            im_pad = sitk.ConstantPad(im_cur,pad_l,pad_u,constant=0)
            im_crop = sitk.Crop(im_pad,crop_l,crop_u)
            sitk.WriteImage(im_crop,normal_path+d+str(i)+s+'.nrrd')

In [3]:
for i in im_num:
    for s in suf1:
        im_T1 = sitk.ReadImage(normal_path+'T1/'+str(i)+s+'.nrrd')
        im_T2 = sitk.ReadImage(normal_path+'T2/'+str(i)+s+'.nrrd')
        im_seg = sitk.ReadImage(normal_path+'seg/'+str(i)+s+'.nrrd')
        print(im_T1.GetSize(),im_T2.GetSize(),im_seg.GetSize(),im_T1.GetSpacing())
        im1_arr = sitk.GetArrayFromImage(im_T1)
        print((min(np.where(im1_arr>0)[0]),max(np.where(im1_arr>0)[0])))
        print((min(np.where(im1_arr>0)[1]),max(np.where(im1_arr>0)[1])))
        print((min(np.where(im1_arr>0)[2]),max(np.where(im1_arr>0)[2])))

((120, 144, 136), (120, 144, 136), (120, 144, 136), (1.0, 1.0, 1.0))
(19, 117)
(14, 131)
(10, 109)
((120, 144, 136), (120, 144, 136), (120, 144, 136), (1.0, 1.0, 1.0))
(19, 117)
(14, 131)
(10, 109)
((120, 144, 136), (120, 144, 136), (120, 144, 136), (1.0, 1.0, 1.0))
(24, 119)
(14, 130)
(11, 104)
((120, 144, 136), (120, 144, 136), (120, 144, 136), (1.0, 1.0, 1.0))
(24, 119)
(14, 130)
(15, 108)
((120, 144, 136), (120, 144, 136), (120, 144, 136), (1.0, 1.0, 1.0))
(27, 114)
(12, 125)
(15, 104)
((120, 144, 136), (120, 144, 136), (120, 144, 136), (1.0, 1.0, 1.0))
(27, 114)
(12, 125)
(15, 104)
((120, 144, 136), (120, 144, 136), (120, 144, 136), (1.0, 1.0, 1.0))
(18, 113)
(19, 131)
(14, 104)
((120, 144, 136), (120, 144, 136), (120, 144, 136), (1.0, 1.0, 1.0))
(18, 113)
(19, 131)
(15, 105)
((120, 144, 136), (120, 144, 136), (120, 144, 136), (0.9999977350234985, 1.0, 0.9999999403953552))
(28, 129)
(15, 130)
(10, 111)
((120, 144, 136), (120, 144, 136), (120, 144, 136), (0.9999977350234985, 1.0, 0